# Reading CSV data

In [38]:
import csv
EDGES = []
with open('vk_links.csv') as csv_file:
    reader = csv.reader(csv_file)
    skip_first = 1
    for row in reader:
        if skip_first == 1:
            skip_first = 0
            continue
        EDGES.append((row[0],row[1]))
    
# Getting unique names
NAMES = set([x for t in EDGES for x in t])

In [34]:
print(GRAPH_DATA)
print(NAMES)

[('ivan', 'nina'), ('nina', 'olga'), ('olga', 'ivan'), ('ivan', 'petr'), ('petr', 'nina'), ('petr', 'olga'), ('pavel', 'nikita'), ('nikita', 'dasha'), ('dasha', 'pavel'), ('pavel', 'nina'), ('pavel', 'masha'), ('masha', 'petr'), ('masha', 'dasha'), ('dasha', 'ivan')]
{'pavel', 'petr', 'ivan', 'dasha', 'nina', 'masha', 'nikita', 'olga'}


# Start building graph

In [35]:
import louvain      # pacaur -S igraph && sudo pip3 install louvain
import igraph as ig

# Creating igraph based on our data
graph = ig.Graph()

# Adding vertices
for key in NAMES:
    graph.add_vertices(key)
    
# Adding edges
graph.add_edges(GRAPH_DATA)

# Clustering data
partitions = louvain.find_partition(graph, louvain.ModularityVertexPartition)
print(partitions)

Clustering with 8 elements and 2 clusters
[0] ivan, nina, olga, petr
[1] pavel, nikita, dasha, masha
